In [11]:
import os
import os.path as op
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from glob import glob
import nibabel as nib
import gzip
import json

In [24]:
class PhilipsPhysioLog:
    """ Reads, converts, and aligns Philips physiology files (SCANPHYSLOG).
    """
 
    def __init__(self, log_file, fmri_file=None, n_dyns=100, sf=500, tr=None):
        """ Initializes PhilipsPhysioLog object. 
        # f: log files
        # fmri_file: None by default, if given pick the tr duration and number
        # sf: spatial frequency of log files
        # tr: None by default, TR in seconds
        """
        self.log_file = log_file
        self.n_dyns = n_dyns
        self.sf = sf
        self.tr = tr
        self.fmri_file = fmri_file

        if fmri_file is not None:
            img = nib.load(fmri_file)
            self.tr = img.header['pixdim'][4]
            self.n_dyns = img.header['dim'][4]           
        
        if self.tr is None:
            raise ValueEror("Please provide a TR")
        
        self.trs = self.tr * self.sf
        
    def load(self):
        
        # read the log file and create markers from log mark column
        with open(self.log_file, 'r') as log_file_in:
            for i, line in enumerate(log_file_in):
                if line[:2] != '##':
                    break
        
            txt = log_file_in.readlines()
            txt = [line.replace('  ', ' ').replace('\n', '') for line in txt if line != '#\n']
            self.markers = np.array([s.split(' ')[9] for s in txt])

        # define start point
        m_start_idx = np.where(self.markers == '0100')[0]
        if len(m_start_idx) == 0:
            m_start_idx = 0
        else:
            m_start_idx = m_start_idx[-1]
        self.m_start_idx = 0
        self.m_start_idx = m_start_idx
        
        # define end point
        m_end_idx = np.where(self.markers == '0020')[0]
        if len(m_end_idx) == 0:
            m_end_idx = len(txt)
        else:
            m_end_idx = m_end_idx[-1]
        self.m_end_idx = m_end_idx
        self.m_end_idx = len(txt)
        
        # define data
        self.dat = np.loadtxt(self.log_file, dtype=int, usecols=np.arange(9))
        # define data lenght
        self.n = self.dat.shape[0]
        # define gradient data
        self.grad = self.dat[:, (6, 7, 8)]

        return self
    
    def align(self, trigger_method='gradient_log', which_grad='y'):
        
        real_triggers = np.where(self.markers == '0200')[0]
        if len(real_triggers) != self.n_dyns:
            print(f"WARNING: expected to find {self.n_dyns} triggers, but found {len(real_triggers)}")
        self.real_triggers = real_triggers
    
    def to_bids(self):
        base_name, _ = op.splitext(self.log_file)
        
        time = np.arange(self.n) / self.sf
        start = time[self.real_triggers[0]]
        time = time - start

        info = {
           "SamplingFrequency": self.sf,
           "StartTime": time[0],
           "Columns": ["cardiac", "respiratory", "trigger"]
        }
        
        with open(f'{base_name}.json', "w") as write_file:
            json.dump(info, write_file, indent=4)
        
        data = self.dat[:, 4:6]
        pulses = np.zeros(self.n)
        pulses[self.real_triggers] = 1
        data = np.c_[data, pulses]
        tsv_out = f'{base_name}.tsv'
        np.savetxt(tsv_out, data, delimiter='\t')
        with open(tsv_out, 'rb') as f_in, gzip.open(tsv_out + '.gz', 'wb') as f_out:
            print(f"Saving to {tsv_out} ...")
            f_out.writelines(f_in)
        os.remove(tsv_out)

In [35]:
log = '/home/shared/2018/visual/pRFgazeMod/bids_data/sub-007/ses-01/func/sub-007_ses-01_task-AttendStimGazeCenter_run-1_physio.log'
fmri = '/home/shared/2018/visual/pRFgazeMod/bids_data/sub-007/ses-01/func/sub-007_ses-01_task-AttendStimGazeCenter_run-1_bold.nii.gz'
sf = 500

In [36]:
a = PhilipsPhysioLog(log_file = log, fmri_file = fmri, sf = 500)
a.load()
a.align(trigger_method='vol_triggers')

In [70]:
trig = a.real_triggers[(a.real_triggers.shape[0] - a.n_dyns):]
np.diff(trig)

array([  658, 40500,   659,   659,   658,   658,   659,   658,   659,
         659,   658,   659,   658,   659,   658,   658,   659,   658,
         659,   658,   659,   658,   659,   658,   659,   658,   659,
         658,   658,   659,   659,   658,   659,   658,   658,   659,
         659,   658,  1317,   658,   659,   659,   658,   659,   658,
         659,   658,   658,   659,   658,   659,   658,   659,   658,
         659,   658,   659,   658,   659,   658,   658,   659,   659,
         658,   659,   658,   658,   659,   659,   658,   658,   659,
         658,   659,   659,   658,   659,  1317,   658,   658,   659,
         658,   659,   658,   659,   658,   659,   658,   659,   658,
         659,   658,   658,   659,   659,   658,   659,   658,   658,
         659,   659,   658,   658,   659,   658,   659,   659,   658,
         659,   658,   659,   658,   658,   659,   658,   659,   658,
         659,   658,   659,   658])

In [86]:
trig[39]

70961

In [89]:
trig

array([  4780,   5438,  45938,  46597,  47256,  47914,  48572,  49231,
        49889,  50548,  51207,  51865,  52524,  53182,  53841,  54499,
        55157,  55816,  56474,  57133,  57791,  58450,  59108,  59767,
        60425,  61084,  61742,  62401,  63059,  63717,  64376,  65035,
        65693,  66352,  67010,  67668,  68327,  68986,  69644,  70961,
        71619,  72278,  72937,  73595,  74254,  74912,  75571,  76229,
        76887,  77546,  78204,  78863,  79521,  80180,  80838,  81497,
        82155,  82814,  83472,  84131,  84789,  85447,  86106,  86765,
        87423,  88082,  88740,  89398,  90057,  90716,  91374,  92032,
        92691,  93349,  94008,  94667,  95325,  95984,  97301,  97959,
        98617,  99276,  99934, 100593, 101251, 101910, 102568, 103227,
       103885, 104544, 105202, 105861, 106519, 107177, 107836, 108495,
       109153, 109812, 110470, 111128, 111787, 112446, 113104, 113762,
       114421, 115079, 115738, 116397, 117055, 117714, 118372, 119031,
      

In [34]:
# read the log file and create markers from log mark column
file = 'projects/pRFgazeMod/stats/lukas/sub-13_ses-mapper1_task-floc_run-1_physio.tsv'

df = pd.read_csv(file, sep="\t")   # read dummy .tsv file into memory
a = df.values  # access the numpy array containing values

In [23]:
np.diff(np.where((a[:,2])==1.0))*2

array([[1318, 1316, 1318, 1316, 1318, 1316, 1318, 1316, 1316, 1318, 1318,
        1316, 1318, 1318, 1314, 1318, 1318, 1318, 1314, 1318, 1316, 1318,
        1318, 1316, 1318, 1316, 1316, 1318, 1316, 1318, 1316, 1318, 1316,
        1318, 1316, 1318, 1316, 1318, 1316, 1318, 1316, 1318, 1316, 1318,
        1316, 1318, 1316, 1316, 1318, 1318, 1316, 1316, 1318, 1318, 1316,
        1318, 1316, 1318, 1316, 1318, 1316, 1316, 1318, 1316, 1318, 1318,
        1316, 1316, 1318, 1316, 1318, 1316, 1318, 1318, 1314, 1318, 1318,
        1316, 1318, 1316, 1316, 1318, 1318, 1316, 1316, 1320, 1314, 1318,
        1318, 1318, 1316, 1316, 1318, 1316, 1316, 1318, 1316, 1318, 1316,
        1318, 1316, 1318, 1316, 1318, 1316, 1318, 1316, 1316, 1318, 1318,
        1316, 1318, 1316, 1316, 1318, 1318, 1316, 1316, 1318, 1316, 1318,
        1318, 1316, 1318, 1316, 1318, 1316, 1316, 1318, 1318, 1316, 1316,
        1318, 1316, 1318, 1316, 1318, 1318, 1316, 1316, 1316, 1318, 1318,
        1316, 1318, 1318, 1314, 1318, 